In [ ]:
import pyforest

from IPython.core.interactiveshell import InteractiveShell # Importing so we can run multiple lines in one cell`==
InteractiveShell.ast_node_interactivity = "all" # Code so multiple lines in one cell can be ran simultaenously`==
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
import warnings
warnings.filterwarnings('ignore')


# ▶  Data Import

In [ ]:
df = pd.read_csv('D:\GitHub Repos\WGU_MSDA\D212_Data Mining\medical_clean.csv')

In [ ]:
df.head()

In [4]:
# Creating a copy of the original dataframe and dropping columns that are not needed for clustering
df_kmeans = df.copy()

# New dataframe for k_means clustering
df_kmeans.drop(['CaseOrder', 'Customer_id', 'Interaction', 'UID', 'City', 
                'Zip', 'Lat', 'Lng', 'Population', 'Area', 'TimeZone',
                'State', 'County', 'Area', 'TimeZone', 'Job', 'Marital', 
                'Gender', 'Soft_drink', 'Initial_admin', 'HighBlood', 'Stroke', 
                'Complication_risk', 'Overweight', 'Arthritis', 'Diabetes', 
                'Hyperlipidemia', 'BackPain', 'Anxiety', 'Allergic_rhinitis', 
                'Reflux_esophagitis', 'Asthma', 'Services', 'Item1', 'Item2',
                'Item3', 'Item4', 'Item5', 'Item6', 'Item7', 'Item8'], axis=1, inplace=True)

In [ ]:
df_kmeans.head()

In [6]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()

df_kmeans['ReAdmis'] = enc.fit_transform(df_kmeans['ReAdmis'])


In [ ]:
# df_kmeans after encoding
df_kmeans.head()

In [ ]:
# Checking is ReAdmis has been encoded  to 0 and 1
df_kmeans['ReAdmis'].unique()

We will now Stardardize our dataframe.  
Reason: K-Means is based on Euclidean distances, and variables with larger scales (e.g., income or population) will dominate the clustering process if not standardized.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_means_columns = df_kmeans.columns    
df_kmeans_scaled = pd.DataFrame(scaler.fit_transform(df_kmeans), columns=df_means_columns)

In [ ]:
df_kmeans_scaled.head()

In [ ]:
df_kmeans_scaled['ReAdmis'].unique()

We have scaled our data and inserted the appropiate column names.

In [ ]:
# Plot correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df_kmeans_scaled.corr(), annot=True, cmap='coolwarm', linewidths=0.5, fmt='.2f')
plt.title("Correlation Heatmap of Features")
plt.show();

We will use 2 variables for the clustering algorithm, namely Age and VitD levels.  
Reason:  
    - Age: While it has some correlation with Additional_charges (0.72), it’s not too high, and age can offer valuable insights for clustering.  
    - VitD_levels: This feature has low correlations with others and could provide interesting insights related to health in the clustering process.

In [ ]:
# Creating a new dataframe with just the variables of interest for k-means clustering
clusterdata = df_kmeans_scaled[['Age', 'VitD_levels', 'ReAdmis']]
clusterdata.head()

In [ ]:
sns.scatterplot(data = clusterdata, x = 'Age', y = 'VitD_levels', hue = 'ReAdmis', palette = 'viridis');

In [ ]:
# Initializing KMeans
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=3, n_init= 25, random_state=42)

k_model.fit(clusterdata)

In [ ]:
# Calculating centroids
centroids = pd.DataFrame(k_model.cluster_centers_, columns = clusterdata.columns)
centroids

In [ ]:
plt.figure(figsize=(12, 10));
ax = sns.scatterplot(data=clusterdata,
                     x='Age',
                     y='VitD_levels',
                     hue=k_model.labels_,
                     palette='colorblind',
                     alpha=0.9,
                     s=150,
                     legend=True)

sns.scatterplot(data=centroids,
                x='Age',
                y='VitD_levels',
                hue=centroids.index,
                palette='colorblind',
                s=900,
                marker='D',
                edgecolor='black',
                legend=False);